In [32]:
import os


In [39]:

# put above lines into a torch dataset that take file_path as init argument
import torch
import soundfile
from torch.utils.data import Dataset
class LibriSampledDataset(Dataset):
    '''
    600 samples of Libri test set
    '''
    def __init__(self, file_path, min_length=0, max_length=100000, transform=None, target_transform=None):
        with open(file_path) as f:
            lines = f.read().strip().split('\n')
        lines = lines
        lines = [l.split(',') for l in lines]
        lines = sorted(lines, key=lambda l: int(l[1]))
        lines = [l[0] for l in lines if min_length <= int(l[1]) < max_length]

        self.lines = lines
        self.utts = []
        for fn  in lines:
            self.utts.append(open(os.path.join('/scratch/f006pq6/datasets/.deep-speaker-wd/samples_deepspeaker/samples/librispeech/single', '%s.csv' % fn)).read().strip().split('\n')[-1])
        self.max_length = max(len(l.split(',')[-1]) for l in self.utts)

        self._transform = transform
        self._target_transform = target_transform



    def __len__(self):
        return len(self.utts)

    def __getitem__(self, index):
        # utts[idx] give path, duration, transcript seperated by comma

        path, dur, target= self.utts[index].split(',') #target is transcript
        audio, rate = soundfile.read(path, dtype='int16')
        assert rate == 16000, '%r sample rate != 16000' % path

        if self._transform is not None:
            audio = self._transform(audio)

        if self._target_transform is not None:
            target = self._target_transform(target)

        return audio, target, path 
       
file_path = '../samples/samples_below_4s_bucket_500_all_minh.txt'
my_dataset = LibriSampledDataset(file_path, min_length=2000, max_length=3000)



In [46]:
my_dataset[6]

(array([25, 41, 60, ..., 44, 33, 77], dtype=int16),
 'the walls of the houses were wet')